In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
torch.__version__

'1.5.0'

In [2]:
BATCH_SIZE = 512
EPOCHS = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data',train = True,download = True,
                                                         transform = transforms.Compose([
                                                             transforms.ToTensor(),
                                                             transforms.Normalize((0.1307,),(0.3081,))
                                                         ])),
                                          batch_size = BATCH_SIZE,shuffle = True)

In [4]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST('data',train = False,transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])),
                                         batch_size = BATCH_SIZE,shuffle = True)

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5)
        self.conv2 = nn.Conv2d(10,20,3)
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out,2,2)
        out = self.conv2(out)
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

In [6]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [7]:
def train(model,device,train_loader,optimizer,epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target = data.to(device),target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output,target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0:
            print('Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
            epoch,batch_idx * len(data),len(train_loader.dataset),
            100. * batch_idx/len(train_loader),loss.item()))

In [8]:
def test(model,device,test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(device),target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output,target,reduction='sum').item()
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss,correct,len(test_loader.dataset),
        100. * correct/len(test_loader.dataset)))

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model,DEVICE,train_loader,optimizer,epoch)
    test(model,DEVICE,test_loader)

Train Epoch:1 [14848/60000 (25%)]	Loss:0.298360
Train Epoch:1 [30208/60000 (50%)]	Loss:0.157732
Train Epoch:1 [45568/60000 (75%)]	Loss:0.107544

Test set: Average loss: 0.0893, Accuracy: 9732/10000 (97%)

Train Epoch:2 [14848/60000 (25%)]	Loss:0.068188
Train Epoch:2 [30208/60000 (50%)]	Loss:0.080535
Train Epoch:2 [45568/60000 (75%)]	Loss:0.055243

Test set: Average loss: 0.0589, Accuracy: 9816/10000 (98%)

Train Epoch:3 [14848/60000 (25%)]	Loss:0.055258
Train Epoch:3 [30208/60000 (50%)]	Loss:0.045292
Train Epoch:3 [45568/60000 (75%)]	Loss:0.036279

Test set: Average loss: 0.0524, Accuracy: 9829/10000 (98%)

Train Epoch:4 [14848/60000 (25%)]	Loss:0.058677
Train Epoch:4 [30208/60000 (50%)]	Loss:0.041679
Train Epoch:4 [45568/60000 (75%)]	Loss:0.053191

Test set: Average loss: 0.0474, Accuracy: 9839/10000 (98%)

Train Epoch:5 [14848/60000 (25%)]	Loss:0.049636
Train Epoch:5 [30208/60000 (50%)]	Loss:0.034009
Train Epoch:5 [45568/60000 (75%)]	Loss:0.046147

Test set: Average loss: 0.0451, Acc